In [2]:
from src.utils.spark_utils import get_spark_session

In [3]:
spark=get_spark_session()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/23 11:52:52 WARN Utils: Your hostname, vivekpanthi-Swift-SF514-55TA, resolves to a loopback address: 127.0.1.1; using 192.168.254.15 instead (on interface wlp0s20f3)
25/11/23 11:52:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/vivekpanthi/DataEngineering/E-commerce-Data-Pipeline/myvenv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/vivekpanthi/.ivy2.5.2/cache
The jars for the packages stored in: /home/vivekpanthi/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c7d5d19b-4dfa-4b6e-bd33-3ba537cdafc8;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: 

In [44]:
df=spark.read.format("delta").load("data/bronze/bronze_orders_raw")

In [45]:
df.show(truncate=False)

+---------+-----------+-------------------+--------+----------+---------------+------------------+----------------+-------------------+-------------+----------------------+--------------+---------------+-----------------+--------------------------------+-------------------+--------------------------------+----------------+----------------------+---------------------------+--------------------------------+------------------------------------+----------------------+--------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------+---------------+
|order_id |customer_id|order_date         |subtotal|tax_amount|discount_amount|total_amount      |payment_method  |created_at         |_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_c

In [4]:
df.createOrReplaceTempView("orders")


In [7]:
spark.sql("""
    SELECT
        *,
        COALESCE(
            try_to_timestamp(order_date, 'yyyy-MM-dd HH:mm:ss'),
            try_to_timestamp(order_date, 'dd-MM-yyyy HH:mm:ss'),
            try_to_timestamp(order_date, 'MM/dd/yyyy HH:mm'),
            try_to_timestamp(order_date, 'MM/dd/yyyy HH:mm:ss'),
            try_to_timestamp(order_date, 'MM/dd/yyyy'),
            try_to_timestamp(order_date, 'yyyy-MM-dd''''T''''HH:mm:ss'),
            try_to_timestamp(order_date, 'yyyy-MM-dd''''T''''HH:mm:ss.SSS'),
            try_to_timestamp(order_date, 'yyyy-MM-dd''''T''''HH:mm:ss.SSSSSS')
        ) AS order_date_ts
    FROM orders
""").show(truncate=False)


IllegalArgumentException: Unknown pattern letter: T

In [ ]:
df=spark.read.format("csv").option("header", True).load("data/landing/customers/customers.csv")

In [25]:
from pyspark.sql.functions import *
df.select(col("order_date")).distinct().show(truncate=False)

+--------------------------+
|order_date                |
+--------------------------+
|04/05/2024 15:52          |
|08-05-2024 08:00:19       |
|2024-03-30T03:59:38.805962|
|2024-02-05T18:42:27.199629|
|29-04-2024 12:49:53       |
|2023-06-15T16:53:27.551851|
|2024-05-13T09:04:34.166427|
|2023-11-18T23:26:49.356169|
|2024-09-10 13:02:51       |
|2023-05-14T06:11:02.015345|
|06/18/2024 23:47          |
|2024-05-11 07:13:51       |
|18-06-2023 18:48:40       |
|2023-08-21 06:37:35       |
|12/12/2023 07:01          |
|2024-09-13 22:37:37       |
|06-01-2023 13:59:30       |
|2024-03-09T14:27:09.818465|
|05/18/2023 18:23          |
|03/26/2024 06:28          |
+--------------------------+
only showing top 20 rows


In [14]:
from pyspark.sql.functions import *

In [17]:
df.count()

1000

In [16]:
df.filter(col("email").isNotNull()).count()

336

In [16]:
BRONZE_TABLE_CUSTOMERS="bronze_ingestion_date"

In [25]:

from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
from typing import Dict, List, Tuple
from datetime import datetime
BRONZE_ORDER_PATH="./data/bronze/bronze_orders_raw"

SILVER_ORDER_PATH="./data/silver/bronze_orders_raw"


spark=get_spark_session()
df=spark.read.format("delta").load("data/bronze/bronze_orders_raw")

# Data Quality Filtering
def filter_invalid_records(df:DataFrame)-> DataFrame:
    
    print("    Filtering invalid records...")

    total_before=df.count()

    df_filtered=df.filter(
        (col("order_id").isNotNull()) & (trim(col("order_id"))!="") &
        (col("customer_id").isNotNull()) & (trim(col("customer_id"))!="") &
        ((col("_dq_has_errors")==False) | (col("_dq_error_count") ==0))
    )

    total_after=df.count()

    filtered_count=total_before-total_after
    print(filtered_count)
    df_filtered.show(truncate=False)
    print(f"      Filtered out {filtered_count} invalid records")
    return df_filtered

def parse_timestamp(date_str):
    """Try multiple date formats"""
    if date_str is None:
        return None
    
    formats = [
        "%Y-%m-%d %H:%M:%S",
        "%d-%m-%Y %H:%M:%S",
        "%d/%m/%Y %H:%M",
        "%m/%d/%Y %H:%M",
        "%m/%d/%Y %H:%M:%S",
        "%m/%d/%Y"
    ]
    
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    
    return None  # Return None if no format matches

parse_timestamp_udf = udf(parse_timestamp, TimestampType())

def standardize_date(df: DataFrame) -> DataFrame:
    print("   Standardizing dates...")
    
    df = df.withColumn(
        "order_date_cleaned",
        parse_timestamp_udf(col("order_date"))
    )
    df = df.withColumn(
        "order_date_cleaned",
        date_format(col("order_date"), "yyyy-MM-dd HH:mm:ss")
    )
    df = df.withColumn(
        "created_at_cleaned",
        parse_timestamp_udf(col("created_at"))
    )
    df = df.withColumn(
        "created_at_cleaned",
        date_format(col("created_at"), "yyyy-MM-dd HH:mm:ss")
    )
    
    df.show(truncate=False)
    return df



In [26]:
df=standardize_date(df)


   Standardizing dates...


25/11/23 12:19:24 ERROR Executor: Exception in task 0.0 in stage 261.0 (TID 3315)
org.apache.spark.SparkDateTimeException: [CAST_INVALID_INPUT] The value '05/16/2024 18:53' of the type "STRING" cannot be cast to "TIMESTAMP" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
== DataFrame ==
"date_format" was called from
jdk.internal.reflect.GeneratedMethodAccessor121.invoke(Unknown Source)

	at org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeErrorInternal(ExecutionErrors.scala:115)
	at org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeErrorInternal$(ExecutionErrors.scala:102)
	at org.apache.spark.sql.errors.ExecutionErrors$.invalidInputInCastToDatetimeErrorInternal(ExecutionErrors.scala:259)
	at org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeError(ExecutionErrors.scala:92)
	at org.apache.spark.sql

DateTimeException: [CAST_INVALID_INPUT] The value '05/16/2024 18:53' of the type "STRING" cannot be cast to "TIMESTAMP" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
== DataFrame ==
"date_format" was called from
jdk.internal.reflect.GeneratedMethodAccessor121.invoke(Unknown Source)


In [65]:
def standardize_numeric_data(df:DataFrame)->DataFrame:
    fields=["subtotal","tax_amount","discount_amount","total_amount"]
    print("     Standardizing numeric fields...")

    for field in fields:
        df=df.withColumn(
            field,
            when(
                (col(field).isNotNull() & (col(field).cast("decimal(10,2)") >=0)),
                expr(f"try_cast({field} AS decimal(10,2))")
            ).otherwise(0.0)
        )


    return df

In [66]:
df=standardize_numeric_data(df)

     Standardizing numeric fields...


In [54]:
df.count()

5100

In [55]:
df.filter((col("subtotal")+col("tax_amount")-col("discount_amount")).cast("decimal(10,2)")!=col("total_amount")).show()

+---------+-----------+-------------------+--------+----------+---------------+------------+----------------+-------------------+-------------+----------------------+--------------+---------------+-----------------+--------------------+-------------------+--------------------+----------------+----------------------+---------------------------+--------------------+--------------------------+----------------------+--------------------------+--------------------+--------------------+---------------+
| order_id|customer_id|         order_date|subtotal|tax_amount|discount_amount|total_amount|  payment_method|         created_at|_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_count|_dq_warning_count|   _dq_error_details|_dq_warning_details| _bronze_record_hash|_dq_is_duplicate|_dq_duplicate_of_batch|_bronze_ingestion_timestamp| _bronze_source_file|_bronze_ingestion_batch_id|_bronze_ingestion_date|_bronze_source_line_number|  _bronze_raw_record|   _bronze_file_hash|_schema_versi

In [ ]:
def validate_amount_consistency(df:DataFrame)->DataFrame:
        print("   Standardizing monetary amounts...")
        df=df.withColumn(
                "calculated_amount",
                (col("subtotal")+
                col("tax_amount")-
                col("discount_amount")).cast("decimal(10,2)")
            )

        df=df.withColumn(
            "amount_consistency",
            abs(col("total_amount")-col("calculated_amount")) <=0.01
        )

        df=df.withColumn(
            "total_amount",
            when(
                col("amount_consistency") == False,
                col("calculated_amount")
            ).otherwise(col("total_amount"))
        )
        df=df.drop("amount_consistency","calculated_amount")
        return df

+---------+-----------+-------------------+--------+----------+---------------+------------+--------------+-------------------+-------------+----------------------+--------------+---------------+-----------------+--------------------+-------------------+--------------------+----------------+----------------------+---------------------------+--------------------+--------------------------+----------------------+--------------------------+--------------------+--------------------+---------------+
| order_id|customer_id|         order_date|subtotal|tax_amount|discount_amount|total_amount|payment_method|         created_at|_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_count|_dq_warning_count|   _dq_error_details|_dq_warning_details| _bronze_record_hash|_dq_is_duplicate|_dq_duplicate_of_batch|_bronze_ingestion_timestamp| _bronze_source_file|_bronze_ingestion_batch_id|_bronze_ingestion_date|_bronze_source_line_number|  _bronze_raw_record|   _bronze_file_hash|_schema_version|


In [ ]:
def standardize_payment_method(df: DataFrame) -> DataFrame:
    """
    Standardize payment method values
    """
    print("   Standardizing payment methods...")
    
    df = df.withColumn(
        "payment_method",
        lower(trim(col("payment_method")))
    )
    
    # Map variations to standard values
    df = df.withColumn(
        "payment_method",
        when(col("payment_method").isin(['credit_card', 'credit card', 'creditcard', 'cc']), 'credit_card')
        .when(col("payment_method").isin(['debit_card', 'debit card', 'debitcard']), 'debit_card')
        .when(col("payment_method").isin(['paypal', 'pay pal']), 'paypal')
        .when(col("payment_method").isin(['gift_card', 'gift card', 'giftcard']), 'gift_card')
        .when(col("payment_method").isin(['cod', 'cash on delivery', 'cash']), 'cash_on_delivery')
        .otherwise('other')
    )
    
    return df

In [77]:
df=standardize_payment_method(df)
df.show()

  → Standardizing payment methods...
+---------+-----------+-------------------+--------+----------+---------------+------------+--------------+-------------------+-------------+----------------------+--------------+---------------+-----------------+--------------------+-------------------+--------------------+----------------+----------------------+---------------------------+--------------------+--------------------------+----------------------+--------------------------+--------------------+--------------------+---------------+-----------------+------------------+
| order_id|customer_id|         order_date|subtotal|tax_amount|discount_amount|total_amount|payment_method|         created_at|_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_count|_dq_warning_count|   _dq_error_details|_dq_warning_details| _bronze_record_hash|_dq_is_duplicate|_dq_duplicate_of_batch|_bronze_ingestion_timestamp| _bronze_source_file|_bronze_ingestion_batch_id|_bronze_ingestion_date|_bronze_sour

In [76]:
df=df.drop("payment_method_cleaned")

In [52]:
from src.utils.spark_utils import get_spark_session
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
from typing import Dict, List, Tuple
from datetime import datetime
from pyspark.sql import Window
import uuid

BRONZE_ORDER_PATH="./data/bronze/bronze_orders_raw"

SILVER_ORDER_PATH="./data/silver/bronze_orders_raw"


spark=get_spark_session()

# Data Quality Filtering
def filter_invalid_records(df:DataFrame)-> DataFrame:
    
    print("    Filtering invalid records...")

    total_before=df.count()

    df_filtered=df.filter(
        (col("order_id").isNotNull()) & (trim(col("order_id"))!="") &
        (col("customer_id").isNotNull()) & (trim(col("customer_id"))!="") &
        (col("_dq_has_errors")==False) | (col("_dq_error_count") ==0)
    )

    total_after=df.count()

    filtered_count=total_before-total_after
    print(f"      Filtered out {filtered_count} invalid records")
    return df_filtered


# standarize datetime
def parse_timestamp(date_str):
    """Try multiple date formats"""
    if date_str is None:
        return None
    
    formats = [
        "%Y-%m-%d %H:%M:%S",
        "%d-%m-%Y %H:%M:%S",
        "%d/%m/%Y %H:%M",
        "%m/%d/%Y %H:%M",
        "%m/%d/%Y %H:%M:%S",
        "%m/%d/%Y"
    ]
    
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    
    return None
parse_time_udf=udf(parse_timestamp,TimestampType())

def standarized_date(df:DataFrame)->DataFrame:
    print("     Standardizing dates...")

    df=df.withColumn(
        "order_date",
        parse_time_udf(col("order_date"))
    )
    df=df.withColumn(
        "order_date",
        date_format(col("order_date"), "yyyy-MM-dd HH:mm:ss")
    )
    df=df.withColumn(
        "created_at",
        parse_time_udf(col("created_at"))
    )
    df=df.withColumn(
        "created_at",
        date_format(col("created_at"), "yyyy-MM-dd HH:mm:ss")
    )
    return df

# standarize numeric 
def standardize_numeric_data(df:DataFrame)->DataFrame:
    fields=["subtotal","tax_amount","discount_amount","total_amount"]
    print("     Standardizing numeric fields...")

    for field in fields:
        df=df.withColumn(
            field,
            when(
                (col(field).isNotNull() & (col(field).cast("decimal(10,2)") >=0)),
                expr(f"try_cast({field} AS decimal(10,2))")
            ).otherwise(0.0)
        )


    return df

def validate_amount_consistency(df:DataFrame)->DataFrame:
        print("   Standardizing monetary amounts...")
        df=df.withColumn(
                "calculated_amount",
                (col("subtotal")+
                col("tax_amount")-
                col("discount_amount")).cast("decimal(10,2)")
            )

        df=df.withColumn(
            "amount_consistency",
            abs(col("total_amount")-col("calculated_amount")) <=0.01
        )

        df=df.withColumn(
            "total_amount",
            when(
                col("amount_consistency") == False,
                col("calculated_amount")
            ).otherwise(col("total_amount"))
        )
        df=df.drop("amount_consistency","calculated_amount")
        return df
def standardize_payment_method(df: DataFrame) -> DataFrame:
    print("   Standardizing payment methods...")
    
    df = df.withColumn(
        "payment_method",
        lower(trim(col("payment_method")))
    )
    
    df = df.withColumn(
        "payment_method",
        when(col("payment_method").isin(['credit_card', 'credit card', 'creditcard', 'cc']), 'credit_card')
        .when(col("payment_method").isin(['debit_card', 'debit card', 'debitcard']), 'debit_card')
        .when(col("payment_method").isin(['paypal', 'pay pal']), 'paypal')
        .when(col("payment_method").isin(['gift_card', 'gift card', 'giftcard']), 'gift_card')
        .when(col("payment_method").isin(['cod', 'cash on delivery', 'cash']), 'cash_on_delivery')
        .otherwise('other')
    )
    
    return df

def deduplicate_orders(df:DataFrame)-> DataFrame:
    print("   Deduplicating records...")
    window_spec=Window.partitionBy("order_id").orderBy(col("created_at").desc())
    df=df.withColumn(
        "row_num",
        row_number().over(window_spec)
    )
    total_before=df.count()
    df_dedup=df.filter(col("row_num")==1).drop("row_num")
    total_after=df_dedup.count()

    duplicates_removed=total_before-total_after

    print(f"      Removed {duplicates_removed} duplicate records")
    
    return df_dedup

def enrich_order_data(df:DataFrame)->DataFrame:
    print("     Enrichment order data")

    df = df.withColumn("order_year", year(col("order_date")))
    df = df.withColumn("order_month", month(col("order_date")))
    df = df.withColumn("order_day", dayofmonth(col("order_date")))
    df = df.withColumn("order_quarter", quarter(col("order_date")))
    df = df.withColumn("order_day_of_week", dayofweek(col("order_date")))
    df = df.withColumn("order_week_of_year", weekofyear(col("order_date")))


    df=df.withColumn(
        "order_day_name",
        when(col("order_day_of_week") == 1, "Sunday")
        .when(col("order_day_of_week") == 2, "Monday")
        .when(col("order_day_of_week") == 3, "Tuesday")
        .when(col("order_day_of_week") == 4, "Wednesday")
        .when(col("order_day_of_week") == 5, "Thursday")
        .when(col("order_day_of_week") == 6, "Friday")
        .when(col("order_day_of_week") == 7, "Saturday")
    )

    df = df.withColumn("order_hour", hour(col("order_date")))

    df = df.withColumn(
        "order_time_period",
        when((col("order_hour") >= 6) & (col("order_hour") < 12), "Morning")
        .when((col("order_hour") >= 12) & (col("order_hour") < 17), "Afternoon")
        .when((col("order_hour") >= 17) & (col("order_hour") < 21), "Evening")
        .otherwise("Night")
    )

    df=df.withColumn(
        "is_weekend",
        col("order_day_of_week").isin([1,7])
    )
    df=df.withColumn(
        "order_value_category",
        when(col("total_amount")>=500,"high_value")
        .when(col("total_amount")>=100,"medium_value")
        .otherwise("low_value")
    )
    df = df.withColumn(
        "discount_percentage",
        when(
            col("subtotal") > 0,
            (col("discount_amount") / col("subtotal") * 100).cast("decimal(10,2)")
        ).otherwise(0.0)
    )
    df = df.withColumn(
        "has_discount",
        col("discount_amount") > 0
    )
    df = df.withColumn(
        "days_since_order",
        datediff(current_date(), col("order_date").cast("date"))
    )
    return df

def select_final_columns(df:DataFrame)->DataFrame:
    return df.select(
            "order_id",
            "customer_id",
            "order_date",
            "subtotal",
            "tax_amount",
            "discount_amount",
            "total_amount",
            "payment_method",
            "created_at",
            "order_year",
            "order_month",
            "order_day",
            "order_quarter",
            "order_day_of_week",
            "order_day_name",
            "order_week_of_year",
            "order_hour",
            "order_time_period",
            "is_weekend"


        )



class SilverOrderPipeline:
    def __init__(self,metadata_manager=None):
        self.batch_id=str(uuid.uuid4())
        self.processing_start=datetime.now()
        self.stats={}
        self.metadata_manager=metadata_manager

    def run(self,bronze_batch_id: str=None):
        print("\n" + "="*70)
        print("SILVER LAYER TRANSFORMATION PIPELINE - ORDERS")
        print("="*70)
        print(f"Batch ID:    {self.batch_id}")
        print(f"Started:     {self.processing_start.strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*70)

        print("\n[1/9] Reading from bronze layer...")
        try:
            bronze_df=spark.read.format("delta").load(BRONZE_ORDER_PATH)
            if bronze_batch_id:
                bronze_df=bronze_df.filter(
                    col("_bronze_ingestion_batch_id") == bronze_batch_id
                )

            else:
                try:
                    silver_df=spark.read.format("delta").load(SILVER_ORDER_PATH)
                    processed_batch=silver_df.select("_bronze_ingestion_batch_id").distinct()


                    bronze_df=bronze_df.join(
                        processed_batch,"_bronze_ingestion_batch_id","left_anti"
                    )
                except:
                    print("     No existing silver data, processing all bronze records")

                records_input=bronze_df.count()
                self.stats["records_input"]=records_input

                print(f"    Read {records_input:,} record from bronze")

                if records_input==0:
                    print("\n No new records to process")
                    return None, self.stats
        except Exception as e:
            print(f"    Error reading bronze: {e}")
            raise

        source_batch_id=bronze_df.select("_bronze_ingestion_batch_id").first()[0]

        print("\n[2/9] Standardizing data...")
        df=bronze_df
        df=standarized_date(df)
        df=standardize_numeric_data(df)
        df=standardize_payment_method(df)
        print("     Stantardization complete")

        print("\n[3/9] Validating amount consistency...")
        df=validate_amount_consistency(df)
        print("   Amount validation complete")

        print("\n[4/9] Deduplicating records...")

        records_before_dedup=df.count()
        df=deduplicate_orders(df) 
        records_after_dedup=df.count()
        self.stats['records_deduplicated'] = records_before_dedup - records_after_dedup
        print("   Deduplication complete")

        print("\n[5/9] Filtering invalid records...")
        records_before_filter=df.count()
        df=filter_invalid_records(df)
        records_after_filter=df.count()

        self.stats["records_filtered"]=records_before_filter-records_after_filter

        print("   Filtering complete")

        print("\n[6/9] Enriching order data...")
        df=enrich_order_data(df)
        print("   Enrichment complete")

        print("\n[7/9] Adding silver metadata...")
        
        # df = add_silver_metadata(df, self.batch_id, source_batch_id)
        
        print("  ✓ Silver metadata added")

        print("\n[8/9] Selecting final columns...")
        
        df_final = select_final_columns(df)
        records_output = df_final.count()
        
        self.stats['records_output'] = records_output
        
        print("  ✓ Final columns selected")
        df_final.write \
            .format("delta") \
            .mode("append") \
            .partitionBy("order_year", "order_month") \
            .option("mergeSchema", "true") \
            .save(SILVER_ORDER_PATH)
        processing_end=datetime.now()
        duration=(processing_end-self.processing_start).total_seconds()
        self.stats['duration_seconds'] = duration
        self.stats['records_per_second'] = records_output / duration if duration > 0 else 0
        dq_improvement = ((records_output / records_input) * 100) if records_input > 0 else 0
        self.stats['data_quality_score'] = dq_improvement
        print("\n" + "="*70)
        print("TRANSFORMATION SUMMARY")
        print("="*70)
        print(f"Status:                ✅ SUCCESS")
        print(f"Duration:              {duration:.2f} seconds")
        print(f"Throughput:            {self.stats['records_per_second']:.0f} records/sec")
        print(f"Records input:         {records_input:,}")
        print(f"Records deduplicated:  {self.stats['records_deduplicated']:,}")
        print(f"Records filtered:      {self.stats['records_filtered']:,}")
        print(f"Records output:        {records_output:,}")
        print(f"Data retention:        {dq_improvement:.1f}%")
        print("="*70 + "\n")
        
        return df_final, self.stats


if __name__ =="__main__":
    pipeline=SilverOrderPipeline()

    df_silver,stats=pipeline.run()


{"ts": "2025-11-23 17:41:23.471", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `_bronze_ingestion_batch_id` cannot be resolved. Did you mean one of the following? [`created_at`, `order_month`, `payment_method`, `customer_id`, `order_date`]. SQLSTATE: 42703", "context": {"file": "jdk.internal.reflect.GeneratedMethodAccessor84.invoke(Unknown Source)", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o10723.select.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `_bronze_ingestion_batch_id` cannot be resolved. Did you mean one of the following? [`created_at`, `order_month`, `payment_method`, `customer_id`, `order_date`]. SQLSTATE: 42703;\n'Project ['_bronze_ingestion_batch_id]\n+- Relati


SILVER LAYER TRANSFORMATION PIPELINE - ORDERS
Batch ID:    423c60e1-b632-47e9-bd40-1f6fb6ec234d
Started:     2025-11-23 17:41:23

[1/9] Reading from bronze layer...
     No existing silver data, processing all bronze records
    Read 5,100 record from bronze

[2/9] Standardizing data...
     Standardizing dates...
     Standardizing numeric fields...
   Standardizing payment methods...
     Stantardization complete

[3/9] Validating amount consistency...
   Standardizing monetary amounts...
   Amount validation complete

[4/9] Deduplicating records...
   Deduplicating records...
      Removed 100 duplicate records
   Deduplication complete

[5/9] Filtering invalid records...
    Filtering invalid records...
      Filtered out 0 invalid records
   Filtering complete

[6/9] Enriching order data...
     Enrichment order data
   Enrichment complete

[7/9] Adding silver metadata...
  ✓ Silver metadata added

[8/9] Selecting final columns...
  ✓ Final columns selected

TRANSFORMATION SUMMAR

In [62]:
df=spark.read.format("csv").option("header",True).load("data/landing/customers/customers.csv")

In [63]:
df.show()

+-----------+----------+---------+--------------------+--------------------+-----------------+----------------+--------------+---------+
|customer_id|first_name|last_name|               email|               phone|registration_date|customer_segment|lifetime_value|is_active|
+-----------+----------+---------+--------------------+--------------------+-----------------+----------------+--------------+---------+
|  CUST93810|    Thomas|    Novak|ronaldwalsh@examp...|    626-898-0860x913|       2023-02-07|        Standard|      10150.49|     True|
|  CUST87397|      ryan|    Burns|                NULL|001-478-209-6213x...|       2024-12-30|         Premium|      10740.29|    False|
|  CUST38893|     laura|    BAKER|robinsonadam@exam...|001-297-758-7390x...|       2020-03-26|        Standard|       5103.76|     True|
|  CUST38221|  jennifer|    Smith|                NULL|                NULL|       2021-09-21|           Basic|      12106.92|    False|
|  CUST80284|   Matthew|  morales|       

## **Silver Customers Transformation**

In [147]:
df=spark.read.format("delta").load("data/bronze/bronze_customers_raw")

In [122]:
df.count()

443

In [81]:
df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- registration_date: string (nullable = true)
 |-- customer_segment: string (nullable = true)
 |-- lifetime_value: string (nullable = true)
 |-- is_active: string (nullable = true)
 |-- _extra_fields: string (nullable = true)
 |-- _dq_completeness_score: decimal(5,2) (nullable = true)
 |-- _dq_has_errors: boolean (nullable = true)
 |-- _dq_error_count: integer (nullable = true)
 |-- _dq_warning_count: integer (nullable = true)
 |-- _dq_error_details: string (nullable = true)
 |-- _dq_warning_details: string (nullable = true)
 |-- _bronze_record_hash: string (nullable = true)
 |-- _dq_is_duplicate: boolean (nullable = true)
 |-- _dq_duplicate_of_batch: string (nullable = true)
 |-- _bronze_ingestion_timestamp: timestamp (nullable = true)
 |-- _bronze_source_file: string (nullable = 

In [148]:
def filter_invalid_records(df:DataFrame)->DataFrame:
    print("    Filtering invalid records...")

    total_before=df.count()
    df_filtered=df.filter(
        (col("customer_id").isNotNull()) & (trim(col("customer_id"))!="")
        & (col("email").isNotNull()) & (trim(col("email"))!="") &
        (col("_dq_has_errors")==False) | (col("_dq_error_count")==0)
    )
    total_after=df_filtered.count()
    filtered_count=total_before-total_after
    print(f"    Filtered out {filtered_count} invalid records")
    return df_filtered
df=filter_invalid_records(df)
df.show()

    Filtering invalid records...
    Filtered out 57 invalid records
+-----------+----------+---------+--------------------+--------------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+-----------------+--------------------+--------------------+----------------+----------------------+---------------------------+--------------------+--------------------------+----------------------+--------------------------+--------------------+--------------------+---------------+
|customer_id|first_name|last_name|               email|               phone|registration_date|customer_segment|lifetime_value|is_active|_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_count|_dq_warning_count|_dq_error_details| _dq_warning_details| _bronze_record_hash|_dq_is_duplicate|_dq_duplicate_of_batch|_bronze_ingestion_timestamp| _bronze_source_file|_bronze_ingestion_batch_id|_bronze_ingestion_date|_br

In [99]:
df.show()

+-----------+----------+---------+--------------------+--------------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+--------------------+--------------------+--------------------+----------------+----------------------+---------------------------+--------------------+--------------------------+----------------------+--------------------------+--------------------+--------------------+---------------+
|customer_id|first_name|last_name|               email|               phone|registration_date|customer_segment|lifetime_value|is_active|_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_count|_dq_warning_count|   _dq_error_details| _dq_warning_details| _bronze_record_hash|_dq_is_duplicate|_dq_duplicate_of_batch|_bronze_ingestion_timestamp| _bronze_source_file|_bronze_ingestion_batch_id|_bronze_ingestion_date|_bronze_source_line_number|  _bronze_raw_record|   _bronze_file_ha

In [149]:
def standard_name(df:DataFrame)->DataFrame:
    fields=["first_name","last_name"]
    for field in fields:
        df=df.withColumn(
            field,
            when(
                col(field).isNotNull() & (trim(col(field))!=""),
                initcap(col(field))
            ).otherwise(lit(None))
        )
    return df

In [150]:
df=standard_name(df)
df.show()

+-----------+----------+---------+--------------------+--------------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+-----------------+--------------------+--------------------+----------------+----------------------+---------------------------+--------------------+--------------------------+----------------------+--------------------------+--------------------+--------------------+---------------+
|customer_id|first_name|last_name|               email|               phone|registration_date|customer_segment|lifetime_value|is_active|_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_count|_dq_warning_count|_dq_error_details| _dq_warning_details| _bronze_record_hash|_dq_is_duplicate|_dq_duplicate_of_batch|_bronze_ingestion_timestamp| _bronze_source_file|_bronze_ingestion_batch_id|_bronze_ingestion_date|_bronze_source_line_number|  _bronze_raw_record|   _bronze_file_hash|_sc

In [118]:
df.show(truncate=False)

+-----------+----------+---------+--------------------------+----------------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+-----------------+------------------------------+--------------------------------+----------------+----------------------+---------------------------+--------------------------------------+------------------------------------+----------------------+--------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------+---------------+-------------------------+------------------------+
|customer_id|first_name|last_name|email                     |phone                 |registration_date|customer_segment|lifetime_value|is_active|_e

In [151]:
def standardize_email(df:DataFrame)-> DataFrame:
    print("   Standardizing emails...")
    email_pattern=r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

    df=df.withColumn(
        "email_valid",
        when(
            col("email").rlike(email_pattern), True
        ).otherwise(False)
    )
    return df
df=standardize_email(df)
df.show(truncate=False)

   Standardizing emails...
+-----------+----------+---------+--------------------------+----------------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+-----------------+------------------------------+--------------------------------+----------------+----------------------+---------------------------+--------------------------------------+------------------------------------+----------------------+--------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------+---------------+-----------+
|customer_id|first_name|last_name|email                     |phone                 |registration_date|customer_segment|lifetime_value|is_active|_extra_fields|

In [126]:
df=df.drop("email_cleaned")

In [129]:
df.filter(col("email_valid")==False).show()

+-----------+----------+---------+-----+-----+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+-----------------+-------------------+-------------------+----------------+----------------------+---------------------------+-------------------+--------------------------+----------------------+--------------------------+------------------+-----------------+---------------+-------------------------+------------------------+-----------+
|customer_id|first_name|last_name|email|phone|registration_date|customer_segment|lifetime_value|is_active|_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_count|_dq_warning_count|_dq_error_details|_dq_warning_details|_bronze_record_hash|_dq_is_duplicate|_dq_duplicate_of_batch|_bronze_ingestion_timestamp|_bronze_source_file|_bronze_ingestion_batch_id|_bronze_ingestion_date|_bronze_source_line_number|_bronze_raw_record|_bronze_file_hash|_schema_version|

In [152]:
def standardize_date(df: DataFrame)->DataFrame:
    print("  → Standardizing dates...")

    df=df.withColumn(
        "registration_date_cleaned",
        coalesce(
            to_date(col("registration_date"), "yyyy-MM-dd"),
            to_date(col("registration_date"), "MM/dd/yyyy"),
            to_date(col("registration_date"), "dd-MM-yyyy"),
            to_date(col("registration_date"), "yyyy-MM-dd'T'HH:mm:ss")
        )
    )
    df = df.withColumn(
        "registration_date_parsed",
        when(col("registration_date_cleaned").isNull() & 
             col("registration_date").isNotNull(), False).otherwise(True)
    )
    
    return df
df=standardize_date(df)
df.show()

  → Standardizing dates...
+-----------+----------+---------+--------------------+--------------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+-----------------+--------------------+--------------------+----------------+----------------------+---------------------------+--------------------+--------------------------+----------------------+--------------------------+--------------------+--------------------+---------------+-----------+-------------------------+------------------------+
|customer_id|first_name|last_name|               email|               phone|registration_date|customer_segment|lifetime_value|is_active|_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_count|_dq_warning_count|_dq_error_details| _dq_warning_details| _bronze_record_hash|_dq_is_duplicate|_dq_duplicate_of_batch|_bronze_ingestion_timestamp| _bronze_source_file|_bronze_ingestion_batch_id|_bron

In [153]:
def standardize_segment(df:DataFrame)->DataFrame:
    print("   Standardizing customer segments...")

    df=df.withColumn(
        "customer_segment_cleaned",
        when(
            lower(trim(col("customer_segment"))).isin(["premium","standard","basic","vip"]),
            lower(trim(col("customer_segment")))
        ).otherwise("standard")
    )
    return df

df=standardize_segment(df)
df.show(truncate=False)

   Standardizing customer segments...
+-----------+----------+---------+--------------------------+----------------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+-----------------+------------------------------+--------------------------------+----------------+----------------------+---------------------------+--------------------------------------+------------------------------------+----------------------+--------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------+---------------+-----------+-------------------------+------------------------+------------------------+
|customer_id|first_name|last_name|email                     |phone     

In [154]:
def standardize_numeric_fields(df: DataFrame) -> DataFrame:

    print("   Standardizing numeric fields...")
    
    df = df.withColumn(
        "lifetime_value_cleaned",
        when(
            col("lifetime_value").isNotNull(),
            col("lifetime_value").cast("decimal(12,2)")
        ).otherwise(lit(0.0).cast("decimal(12,2)"))
    )
    

    df = df.withColumn(
        "lifetime_value_cleaned",
        when(
            col("lifetime_value_cleaned") < 0,
            lit(0.0).cast("decimal(12,2)")
        ).otherwise(col("lifetime_value_cleaned"))
    )
    
    return df
df=standardize_numeric_fields(df)
df.show(truncate=False)

   Standardizing numeric fields...
+-----------+----------+---------+--------------------------+----------------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+-----------------+------------------------------+--------------------------------+----------------+----------------------+---------------------------+--------------------------------------+------------------------------------+----------------------+--------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------+---------------+-----------+-------------------------+------------------------+------------------------+----------------------+
|customer_id|first_name|last_name|email            

In [155]:
def standardize_boolean_fields(df: DataFrame) -> DataFrame:

    print("   Standardizing boolean fields...")
    
    df = df.withColumn(
        "is_active_cleaned",
        when(
            lower(trim(col("is_active"))).isin(['true', '1', 'y', 'yes']),
            True
        ).when(
            lower(trim(col("is_active"))).isin(['false', '0', 'n', 'no']),
            False
        ).otherwise(None)
    )
    
    return df
df=standardize_boolean_fields(df)
df.show(truncate=False)

   Standardizing boolean fields...
+-----------+----------+---------+--------------------------+----------------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+-----------------+------------------------------+--------------------------------+----------------+----------------------+---------------------------+--------------------------------------+------------------------------------+----------------------+--------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------+---------------+-----------+-------------------------+------------------------+------------------------+----------------------+-----------------+
|customer_id|first_name|last_name

In [157]:
df=df.withColumn(
        "phone_digits",
        regexp_replace(col("phone"), "[^0-9]", "")
)
df.show()
df.withColumn(
        "phone_digits",
        when(length(col("phone_digits")) >= 10, substring(col("phone_digits"), -10, 10))
        .otherwise(None)
    ).show()

+-----------+----------+---------+--------------------+--------------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+-----------------+--------------------+--------------------+----------------+----------------------+---------------------------+--------------------+--------------------------+----------------------+--------------------------+--------------------+--------------------+---------------+-----------+-------------------------+------------------------+------------------------+----------------------+-----------------+------------------+
|customer_id|first_name|last_name|               email|               phone|registration_date|customer_segment|lifetime_value|is_active|_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_count|_dq_warning_count|_dq_error_details| _dq_warning_details| _bronze_record_hash|_dq_is_duplicate|_dq_duplicate_of_batch|_bronze_ingestion_times

In [161]:
def standardize_phone_us_format(df: DataFrame) -> DataFrame:
    print("   Standardizing phone numbers to (XXX) XXX-XXXX format...")

    df = df.withColumn(
        "phone_digits",
        regexp_replace(col("phone"), "[^0-9]", "")
    )

    df = df.withColumn(
        "phone_digits",
        when(length(col("phone_digits")) >= 10, substring(col("phone_digits"), -10, 10))
        .otherwise(None)
    )

    df = df.withColumn(
        "phone",
        when(col("phone_digits").isNotNull(),
             concat(
                 lit("("), substring(col("phone_digits"), 1, 3), lit(") "),
                 substring(col("phone_digits"), 4, 3), lit("-"),
                 substring(col("phone_digits"), 7, 4)
             )
        ).otherwise(None)
    )


    df = df.drop("phone_digits")

    return df
df=standardize_phone_us_format(df)
df.show()

   Standardizing phone numbers to (XXX) XXX-XXXX format...
+-----------+----------+---------+--------------------+--------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+-----------------+--------------------+--------------------+----------------+----------------------+---------------------------+--------------------+--------------------------+----------------------+--------------------------+--------------------+--------------------+---------------+-----------+-------------------------+------------------------+------------------------+----------------------+-----------------+
|customer_id|first_name|last_name|               email|         phone|registration_date|customer_segment|lifetime_value|is_active|_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_count|_dq_warning_count|_dq_error_details| _dq_warning_details| _bronze_record_hash|_dq_is_duplicate|_dq_duplicate_of_b

In [162]:
def deduplicate_customers(df:DataFrame)->DataFrame:
    print("   Deduplicating records...")

    window_spec=Window.partitionBy("customer_id").orderBy(col("_bronze_ingestion_timestamp").desc())
    df=df.withColumn(
        "row_num",
        row_number().over(window_spec)
    )
    total_before=df.count()
    df_duped=df.filter(col("row_num")==1).drop("row_num")
    total_after=df_duped.count()

    duplicates_removed=total_before-total_after

    print(f"      Removed {duplicates_removed} duplicate records")
    
    return df_duped

df=deduplicate_customers(df)
df.show(truncate=False)

   Deduplicating records...
      Removed 0 duplicate records
+-----------+----------+---------+------------------------------+--------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+-----------------+------------------------------+--------------------------------+----------------+----------------------+---------------------------+--------------------------------------+------------------------------------+----------------------+--------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------+---------------+-----------+-------------------------+------------------------+------------------------+----------------------+-----------------+
|cust

In [160]:
df.count()

443

In [169]:
def enrich_customers_data(df:DataFrame)->DataFrame:
    print("   Enriching customer data...")
    df=df.withColumn(
        "full_name",
        when(
            col("first_name").isNotNull() & col("last_name").isNotNull(),
            concat(col("first_name"), lit(" "), col("last_name"))
        ).when(
            col("first_name").isNotNull(),
            col("first_name")
        ).when(
            col("last_name").isNotNull(),
            col("last_name")
        ).otherwise(None)
    )

    df=df.withColumn(
        "customer_age_days",
        when(
            col("registration_date_cleaned").isNotNull(),
            datediff(current_date(),col("registration_date_cleaned"))
        ).otherwise(None)
    )
    df = df.withColumn(
        "customer_tier",
        when(col("lifetime_value_cleaned") >= 5000, "platinum")
        .when(col("lifetime_value_cleaned") >= 2000, "gold")
        .when(col("lifetime_value_cleaned") >= 500, "silver")
        .otherwise("bronze")
    )

    df=df.withColumn(
        "email_domain",
        when(
            col("email").isNotNull(),
            regexp_extract(col("email"),r'@(.+)$',1)
        ).otherwise(None)
    )

    df = df.withColumn(
        "has_verified_contact",
        (col("email_valid") == True) | (col("phone").isNotNull())
    )
    return df
df=enrich_customers_data(df)
df.show(truncate=False)
    

   Enriching customer data...
+-----------+----------+---------+------------------------------+--------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+-----------------+------------------------------+--------------------------------+----------------+----------------------+---------------------------+--------------------------------------+------------------------------------+----------------------+--------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------+---------------+-----------+-------------------------+------------------------+------------------------+----------------------+-----------------+---------------+-----------------+----

In [170]:
def select_final_columns(df:DataFrame)->DataFrame:
    return df.select(
        "customer_id",
        "first_name",
        "last_name",
        "full_name",
        "email",
        "email_valid",
        "email_domain",
        "phone",
        "has_verified_contact",
        col("registration_date_cleaned").alias("registration_date"),
        "registration_date_parsed",
        "customer_age_days",
        col("customer_segment_cleaned").alias("customer_segment"),
        col("lifetime_value_cleaned").alias("lifetime_value"),
        "customer_tier",
        col("is_active_cleaned").alias("is_active")
    )
df=select_final_columns(df)
df.show(truncate=False)

+-----------+----------+---------+---------------+------------------------------+-----------+------------+--------------+--------------------+-----------------+------------------------+-----------------+----------------+--------------+-------------+---------+
|customer_id|first_name|last_name|full_name      |email                         |email_valid|email_domain|phone         |has_verified_contact|registration_date|registration_date_parsed|customer_age_days|customer_segment|lifetime_value|customer_tier|is_active|
+-----------+----------+---------+---------------+------------------------------+-----------+------------+--------------+--------------------+-----------------+------------------------+-----------------+----------------+--------------+-------------+---------+
|CUST10089  |Kelly     |Shaw     |Kelly Shaw     |ericharding@example.org       |true       |example.org |NULL          |true                |2020-08-06       |true                    |1935             |standard        |

In [69]:
df.count()

500

In [73]:
df.filter(col("email").isNull()).show(truncate=False)

+-----------+----------+---------+-----+----------------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+--------------------------------+------------------------------+--------------------------------+----------------+----------------------+---------------------------+--------------------------------------+------------------------------------+----------------------+--------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------+---------------+
|customer_id|first_name|last_name|email|phone                 |registration_date|customer_segment|lifetime_value|is_active|_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_count|_dq_warning_count|_dq_error_details           

In [71]:
df.show(truncate=False)

+-----------+----------+---------+-------------------------+----------------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+--------------------------------+------------------------------+--------------------------------+----------------+----------------------+---------------------------+--------------------------------------+------------------------------------+----------------------+--------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------+---------------+
|customer_id|first_name|last_name|email                    |phone                 |registration_date|customer_segment|lifetime_value|is_active|_extra_fields|_dq_completeness_score|_dq_

In [180]:
df=spark.read.format("delta").load("data/bronze/bronze_order_items_raw")

In [181]:
df.show()

+----------+---------+----------+--------------------+-------------+--------+----------+----------+---------+-------------+----------------------+--------------+---------------+-----------------+--------------------+-------------------+--------------------+----------------+----------------------+---------------------------+--------------------+--------------------------+----------------------+--------------------------+--------------------+--------------------+---------------+
|   item_id| order_id|product_id|        product_name|     category|quantity|unit_price|line_total|      sku|_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_count|_dq_warning_count|   _dq_error_details|_dq_warning_details| _bronze_record_hash|_dq_is_duplicate|_dq_duplicate_of_batch|_bronze_ingestion_timestamp| _bronze_source_file|_bronze_ingestion_batch_id|_bronze_ingestion_date|_bronze_source_line_number|  _bronze_raw_record|   _bronze_file_hash|_schema_version|
+----------+---------+----------+---

In [182]:

for c in df.columns:
    if c.startswith("_"):
        df=df.drop(c)

df.show()

+----------+---------+----------+--------------------+-------------+--------+----------+----------+---------+
|   item_id| order_id|product_id|        product_name|     category|quantity|unit_price|line_total|      sku|
+----------+---------+----------+--------------------+-------------+--------+----------+----------+---------+
|ITM3578673|ORD501936|  PROD3214|         up_PROD3214|  Electronics|       1|    360.46|    360.46|SKU-22768|
|ITM5286957|ORD118786|  PROD3914|      STUFF_PROD3914|  Electronics|       5|    335.56|    1677.8|SKU-51908|
|ITM2439596|ORD118786|  PROD6272|     itself_prod6272|         NULL|       4|     173.4|     693.6|SKU-15602|
|ITM7669589|ORD118786|  PROD8295|   American_PROD8295|         NULL|       1|    393.33|    393.33|     NULL|
|ITM5448799|ORD118786|  PROD6784|      focus_prod6784|  ELECTRONICS|       5|     450.6|    2253.0|SKU-73408|
|ITM5915693|ORD118786|  PROD7456|     SEASON_PROD7456|Home & Garden|       4|    331.32|   1325.28|SKU-36001|
|ITM175285

In [175]:
from src.utils.spark_utils import get_spark_session
from src.silver.silver_metadata import add_silver_metadata
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
from typing import Dict, List, Tuple
from datetime import datetime
from pyspark.sql import Window
import uuid

BRONZE_ORDER_PATH="./data/bronze/bronze_customers_raw"

SILVER_ORDER_PATH="./data/silver/silver_customers"

spark=get_spark_session()

def filter_invalid_records(df:DataFrame)->DataFrame:
    print("    Filtering invalid records...")

    total_before=df.count()
    df_filtered=df.filter(
        (col("customer_id").isNotNull()) & (trim(col("customer_id"))!="")
        & (col("email").isNotNull()) & (trim(col("email"))!="") &
        (col("_dq_has_errors")==False) | (col("_dq_error_count")==0)
    )
    total_after=df_filtered.count()
    filtered_count=total_before-total_after
    print(f"    Filtered out {filtered_count} invalid records")
    return df_filtered

def standard_name(df:DataFrame)->DataFrame:
    fields=["first_name","last_name"]
    for field in fields:
        df=df.withColumn(
            field,
            when(
                col(field).isNotNull() & (trim(col(field))!=""),
                initcap(col(field))
            ).otherwise(lit(None))
        )
    return df

def standardize_email(df:DataFrame)-> DataFrame:
    print("   Standardizing emails...")
    email_pattern=r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

    df=df.withColumn(
        "email_valid",
        when(
            col("email").rlike(email_pattern), True
        ).otherwise(False)
    )
    return df

def standardize_date(df: DataFrame)->DataFrame:
    print("  → Standardizing dates...")

    df=df.withColumn(
        "registration_date_cleaned",
        coalesce(
            to_date(col("registration_date"), "yyyy-MM-dd"),
            to_date(col("registration_date"), "MM/dd/yyyy"),
            to_date(col("registration_date"), "dd-MM-yyyy"),
            to_date(col("registration_date"), "yyyy-MM-dd'T'HH:mm:ss")
        )
    )
    df = df.withColumn(
        "registration_date_parsed",
        when(col("registration_date_cleaned").isNull() & 
             col("registration_date").isNotNull(), False).otherwise(True)
    )
    
    return df

def standardize_segment(df:DataFrame)->DataFrame:
    print("   Standardizing customer segments...")

    df=df.withColumn(
        "customer_segment_cleaned",
        when(
            lower(trim(col("customer_segment"))).isin(["premium","standard","basic","vip"]),
            lower(trim(col("customer_segment")))
        ).otherwise("standard")
    )
    return df

def standardize_numeric_fields(df: DataFrame) -> DataFrame:

    print("   Standardizing numeric fields...")
    
    df = df.withColumn(
        "lifetime_value_cleaned",
        when(
            col("lifetime_value").isNotNull(),
            col("lifetime_value").cast("decimal(12,2)")
        ).otherwise(lit(0.0).cast("decimal(12,2)"))
    )
    

    df = df.withColumn(
        "lifetime_value_cleaned",
        when(
            col("lifetime_value_cleaned") < 0,
            lit(0.0).cast("decimal(12,2)")
        ).otherwise(col("lifetime_value_cleaned"))
    )
    
    return df


def standardize_boolean_fields(df: DataFrame) -> DataFrame:

    print("   Standardizing boolean fields...")
    
    df = df.withColumn(
        "is_active_cleaned",
        when(
            lower(trim(col("is_active"))).isin(['true', '1', 'y', 'yes']),
            True
        ).when(
            lower(trim(col("is_active"))).isin(['false', '0', 'n', 'no']),
            False
        ).otherwise(None)
    )
    
    return df


def standardize_phone_us_format(df: DataFrame) -> DataFrame:
    print("   Standardizing phone numbers to (XXX) XXX-XXXX format...")

    df = df.withColumn(
        "phone_digits",
        regexp_replace(col("phone"), "[^0-9]", "")
    )

    df = df.withColumn(
        "phone_digits",
        when(length(col("phone_digits")) >= 10, substring(col("phone_digits"), -10, 10))
        .otherwise(None)
    )

    df = df.withColumn(
        "phone",
        when(col("phone_digits").isNotNull(),
             concat(
                 lit("("), substring(col("phone_digits"), 1, 3), lit(") "),
                 substring(col("phone_digits"), 4, 3), lit("-"),
                 substring(col("phone_digits"), 7, 4)
             )
        ).otherwise(None)
    )


    df = df.drop("phone_digits")

    return df


def deduplicate_customers(df:DataFrame)->DataFrame:
    print("   Deduplicating records...")

    window_spec=Window.partitionBy("customer_id").orderBy(col("_bronze_ingestion_timestamp").desc())
    df=df.withColumn(
        "row_num",
        row_number().over(window_spec)
    )
    total_before=df.count()
    df_duped=df.filter(col("row_num")==1).drop("row_num")
    total_after=df_duped.count()

    duplicates_removed=total_before-total_after

    print(f"      Removed {duplicates_removed} duplicate records")
    
    return df_duped


def enrich_customers_data(df:DataFrame)->DataFrame:
    print("   Enriching customer data...")
    df=df.withColumn(
        "full_name",
        when(
            col("first_name").isNotNull() & col("last_name").isNotNull(),
            concat(col("first_name"), lit(" "), col("last_name"))
        ).when(
            col("first_name").isNotNull(),
            col("first_name")
        ).when(
            col("last_name").isNotNull(),
            col("last_name")
        ).otherwise(None)
    )

    df=df.withColumn(
        "customer_age_days",
        when(
            col("registration_date_cleaned").isNotNull(),
            datediff(current_date(),col("registration_date_cleaned"))
        ).otherwise(None)
    )
    df = df.withColumn(
        "customer_tier",
        when(col("lifetime_value_cleaned") >= 5000, "platinum")
        .when(col("lifetime_value_cleaned") >= 2000, "gold")
        .when(col("lifetime_value_cleaned") >= 500, "silver")
        .otherwise("bronze")
    )

    df=df.withColumn(
        "email_domain",
        when(
            col("email").isNotNull(),
            regexp_extract(col("email"),r'@(.+)$',1)
        ).otherwise(None)
    )

    df = df.withColumn(
        "has_verified_contact",
        (col("email_valid") == True) | (col("phone").isNotNull())
    )
    return df



def select_final_columns(df:DataFrame)->DataFrame:
    return df.select(
        "customer_id",
        "first_name",
        "last_name",
        "full_name",
        "email",
        "email_valid",
        "email_domain",
        "phone",
        "has_verified_contact",
        col("registration_date_cleaned").alias("registration_date"),
        "registration_date_parsed",
        "customer_age_days",
        col("customer_segment_cleaned").alias("customer_segment"),
        col("lifetime_value_cleaned").alias("lifetime_value"),
        "customer_tier",
        col("is_active_cleaned").alias("is_active"),
        "_silver_processing_timestamp",
        "_silver_processing_date",
        "_silver_batch_id",
        "_silver_source_batch_id",
        "_silver_record_hash",
    )


    

class SilverPipeline:
    def __init__(self,metadata_manager=None):
        self.batch_id=str(uuid.uuid4())
        self.processing_start=datetime.now()
        self.stats={}
        self.metadata_manager=metadata_manager

    def run(self,bronze_batch_id: str=None):
        print("\n" + "="*70)
        print("SILVER LAYER TRANSFORMATION PIPELINE - CUSTOMERS")
        print("="*70)
        print(f"Batch ID:    {self.batch_id}")
        print(f"Started:     {self.processing_start.strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*70)

        print("\n[1/8] Reading from bronze layer...")
        try:
            bronze_df=spark.read.format("delta").load(BRONZE_ORDER_PATH)
            if bronze_batch_id:
                bronze_df=bronze_df.filter(
                    col("_bronze_ingestion_batch_id") == bronze_batch_id
                )

            else:
                try:
                    silver_df=spark.read.format("delta").load(SILVER_ORDER_PATH)
                    processed_batch=silver_df.select("_bronze_ingestion_batch_id").distinct()


                    bronze_df=bronze_df.join(
                        processed_batch,"_bronze_ingestion_batch_id","left_anti"
                    )
                except:
                    print("     No existing silver data, processing all bronze records")

                records_input=bronze_df.count()
                self.stats["records_input"]=records_input

                print(f"    Read {records_input:,} record from bronze")

                if records_input==0:
                    print("\n No new records to process")
                    return None, self.stats
        except Exception as e:
            print(f"    Error reading bronze: {e}")
            raise

        source_batch_id=bronze_df.select("_bronze_ingestion_batch_id").first()[0]

        print("\n[2/8] Standardizing data...")
        df=bronze_df
        df=standard_name(df)
        df=standardize_email(df)
        df=standardize_date(df)
        df=standardize_segment(df)
        df=standardize_numeric_fields(df)
        df=standardize_boolean_fields(df)
        df=standardize_phone_us_format(df)
        print("     Stantardization complete")

        print("\n[3/8] Deduplicating records...")

        records_before_dedup=df.count()
        df=deduplicate_customers(df) 
        records_after_dedup=df.count()
        self.stats['records_deduplicated'] = records_before_dedup - records_after_dedup
        print("   Deduplication complete")

        print("\n[4/9] Filtering invalid records...")
        records_before_filter=df.count()
        df=filter_invalid_records(df)
        records_after_filter=df.count()

        self.stats["records_filtered"]=records_before_filter-records_after_filter

        print("   Filtering complete")

        print("\n[5/8] Enriching order data...")
        df=enrich_customers_data(df)
        print("   Enrichment complete")

        print("\n[6/8] Adding silver metadata...")
        
        df = add_silver_metadata(df, self.batch_id, source_batch_id)
        
        print("  ✓ Silver metadata added")

        print("\n[7/8] Selecting final columns...")
        
        df_final = select_final_columns(df)
        records_output = df_final.count()
        
        self.stats['records_output'] = records_output
        
        print("  ✓ Final columns selected")

        print("\n[8/8] Writing to silver layer...")
        df_final.write \
            .format("delta") \
            .mode("append") \
            .partitionBy("_silver_processing_date") \
            .option("mergeSchema", "true") \
            .save(SILVER_ORDER_PATH)
        processing_end=datetime.now()
        duration=(processing_end-self.processing_start).total_seconds()
        self.stats['duration_seconds'] = duration
        self.stats['records_per_second'] = records_output / duration if duration > 0 else 0
        dq_improvement = ((records_output / records_input) * 100) if records_input > 0 else 0
        self.stats['data_quality_score'] = dq_improvement
        print("\n" + "="*70)
        print("TRANSFORMATION SUMMARY")
        print("="*70)
        print(f"Status:                ✅ SUCCESS")
        print(f"Duration:              {duration:.2f} seconds")
        print(f"Throughput:            {self.stats['records_per_second']:.0f} records/sec")
        print(f"Records input:         {records_input:,}")
        print(f"Records deduplicated:  {self.stats['records_deduplicated']:,}")
        print(f"Records filtered:      {self.stats['records_filtered']:,}")
        print(f"Records output:        {records_output:,}")
        print(f"Data retention:        {dq_improvement:.1f}%")
        print("="*70 + "\n")
        
        return df_final, self.stats
    

if __name__ =="__main__":
    pipeline=SilverPipeline()
    df_silver,stats=pipeline.run()


SILVER LAYER TRANSFORMATION PIPELINE - CUSTOMERS
Batch ID:    d0643ad7-a7b6-4c7e-8bab-87e891f15f1f
Started:     2025-11-23 21:38:06

[1/8] Reading from bronze layer...
     No existing silver data, processing all bronze records
    Read 500 record from bronze

[2/8] Standardizing data...
   Standardizing emails...
  → Standardizing dates...
   Standardizing customer segments...
   Standardizing numeric fields...
   Standardizing boolean fields...
   Standardizing phone numbers to (XXX) XXX-XXXX format...
     Stantardization complete

[3/8] Deduplicating records...
   Deduplicating records...
      Removed 0 duplicate records
   Deduplication complete

[4/9] Filtering invalid records...
    Filtering invalid records...
    Filtered out 57 invalid records
   Filtering complete

[5/8] Enriching order data...
   Enriching customer data...
   Enrichment complete

[6/8] Adding silver metadata...
  ✓ Silver metadata added

[7/8] Selecting final columns...
  ✓ Final columns selected

[8/8] W

AnalysisException: [DELTA_PARTITION_COLUMN_NOT_FOUND] Partition column `_silver_processing_date` not found in schema [customer_id, first_name, last_name, full_name, email, email_valid, email_domain, phone, has_verified_contact, registration_date, registration_date_parsed, customer_age_days, customer_segment, lifetime_value, customer_tier, is_active]